Information comes from two sources:
1. the CSV file containing the schedule for the grid search
2. the DeepRacer evaluation JSON files 

In [6]:
import pandas as pd
import glob
import json

In [7]:
vers = 'v1'
datestr = '030821'

Load the CSV file containing the schedule for the grid search.

In [8]:
deep_racer_grid = pd.read_csv('deep_racer_grid.csv')
deep_racer_grid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Index                100 non-null    int64  
 1   batch_size           100 non-null    int64  
 2   epochs               100 non-null    int64  
 3   learning_rate        100 non-null    float64
 4   entropy              100 non-null    float64
 5   discount_factor      100 non-null    float64
 6   loss_type            100 non-null    object 
 7   experience_episodes  100 non-null    int64  
 8   car                  100 non-null    object 
 9   id                   100 non-null    object 
 10  assign               100 non-null    object 
dtypes: float64(3), int64(4), object(4)
memory usage: 8.7+ KB


In [9]:
# Retain only pertinent columns in the data frame.
# I am dropping 'car' because the same car is always used (for now).
deep_racer_grid = deep_racer_grid[['id', 'Index', 'batch_size', 'epochs', 'learning_rate', 'entropy', 'discount_factor',
                                   'loss_type', 'experience_episodes', 'assign']]
deep_racer_grid.head()

,id,Index,batch_size,epochs,learning_rate,entropy,discount_factor,loss_type,experience_episodes,assign
0,bhi1,1,64,3,0.0003,0.0010,0.9999,mse,20,Joe
1,bhi2,2,32,10,0.0010,0.0010,0.9900,huber,10,Joe
2,bhi3,3,32,8,0.0003,0.0001,0.9900,mse,20,Joe
3,bhi4,4,32,10,0.0010,0.0100,0.9900,mse,10,Joe
4,bhi5,5,64,10,0.0003,0.0100,0.9999,huber,20,Joe


In [10]:
# Create list of BHI ids.
bhi_id_list = deep_racer_grid['id'].unique().tolist()

In [11]:
# Initialize temporary output data frame.
tmp_out_df = None

# Loop over BHI ids.
for bhi_id in bhi_id_list:

    # Make a list of JSON files that match the current id.
    glob_list = glob.glob(f'evaluations\{bhi_id}\metrics\evaluation\*.json')

    # Reference: https://www.geeksforgeeks.org/read-json-file-using-python/
    if len(glob_list) > 0:

        # Loop over JSON files that we found.
        for j in glob_list:
            # Open JSON file 
            f = open(j)

            # returns JSON object as a dictionary 
            data = json.load(f) 

            # Iterating through the json list.
            for i, d in enumerate(data['metrics']):
                tmp_df = pd.Series(d).to_frame().transpose()
                if i == 0:
                    id_df = tmp_df
                else:
                    id_df = id_df.append(tmp_df)

            # Closing file 
            f.close()

        # Convert episode_status to numeric representation.
        id_df['lap_complete'] = id_df['episode_status'].apply(lambda x: 1 if x=='Lap complete' else 0)
        id_df.drop('episode_status',axis=1)

        # Average metrics across trials within current id.
        # Convert Series to DataFrame with Series indices as column names.
        # Reference: https://stackoverflow.com/a/40224468
#         mean_out_series = id_df.mean(axis=0)
#         mean_out_df = pd.DataFrame(mean_out_series).transpose()

        # Add 'id' as the index.
#         col_names = mean_out_df.columns.tolist()
#         mean_out_df['id'] = bhi_id
        id_df.insert(0,'id',bhi_id)
        #mean_out_df.set_index('id',inplace=True)
        
        # Put the 'id' column first.
        mean_out_df = id_df[['id',
                                   'off_track_count',
                                   'reversed_count',
                                   'trial',
                                   'reset_count',
                                   'completion_percentage',
                                   'elapsed_time_in_milliseconds',
                                   'immobilized_count',
                                   'metric_time',
                                   'crash_count',
                                   'start_time',
                                   'lap_complete']]

        # Add average metrics to the temporary output data frame.
        if tmp_out_df is None:
            tmp_out_df = mean_out_df
        else:
            tmp_out_df = tmp_out_df.append(mean_out_df, ignore_index=True)

In [12]:
# Now merge the temporary output data frame with the main output data frame.
deep_racer_grid = pd.merge(deep_racer_grid, tmp_out_df, on='id', how='outer')
deep_racer_grid

,id,Index,batch_size,epochs,learning_rate,entropy,discount_factor,loss_type,experience_episodes,assign,...,reversed_count,trial,reset_count,completion_percentage,elapsed_time_in_milliseconds,immobilized_count,metric_time,crash_count,start_time,lap_complete
0,bhi1,1,64,3,0.000300,0.0010,0.9999,mse,20,Joe,...,0,1,1,27,4101,0,7966,0,3865,0.0
1,bhi1,1,64,3,0.000300,0.0010,0.9999,mse,20,Joe,...,0,2,1,29,4652,0,13167,0,8515,0.0
2,bhi1,1,64,3,0.000300,0.0010,0.9999,mse,20,Joe,...,0,3,1,93,12075,0,25563,0,13488,0.0
3,bhi1,1,64,3,0.000300,0.0010,0.9999,mse,20,Joe,...,0,4,0,100,13723,0,39641,0,25918,1.0
4,bhi1,1,64,3,0.000300,0.0010,0.9999,mse,20,Joe,...,0,5,1,95,12906,0,53038,0,40132,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,bhi96,96,64,8,0.000003,0.0010,0.9999,mse,10,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,bhi97,97,128,3,0.000030,0.0100,0.9999,huber,50,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
357,bhi98,98,64,10,0.000003,0.0001,0.9990,huber,10,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
358,bhi99,99,64,3,0.000030,0.0001,0.9990,huber,10,Dan L,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# Write to CSV file.
outFile = 'DeepRacer Grid - ' + vers + '.' + datestr + '.csv'
deep_racer_grid.to_csv(outFile,index=False)

In [15]:
deep_racer_grid[~deep_racer_grid.trial.isna()].to_csv('eval_results.csv')

In [ ]:
deep_racer_grid[completion_percentage]

In [19]:
df = deep_racer_grid[~deep_racer_grid.trial.isna()]
pd.get_dummies(df, columns=['batch_size', 'epochs','learning_rate','entropy','discount_factor','loss_type','experience_episodes'])

,id,Index,assign,off_track_count,reversed_count,trial,reset_count,completion_percentage,elapsed_time_in_milliseconds,immobilized_count,...,entropy_0.001,entropy_0.01,discount_factor_0.99,discount_factor_0.9990000000000001,discount_factor_0.9999,loss_type_huber,loss_type_mse,experience_episodes_10,experience_episodes_20,experience_episodes_50
0,bhi1,1,Joe,1,0,1,1,27,4101,0,...,1,0,0,0,1,0,1,0,1,0
1,bhi1,1,Joe,1,0,2,1,29,4652,0,...,1,0,0,0,1,0,1,0,1,0
2,bhi1,1,Joe,1,0,3,1,93,12075,0,...,1,0,0,0,1,0,1,0,1,0
3,bhi1,1,Joe,0,0,4,0,100,13723,0,...,1,0,0,0,1,0,1,0,1,0
4,bhi1,1,Joe,1,0,5,1,95,12906,0,...,1,0,0,0,1,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,bhi95,95,Dan L,1,0,1,1,31,17281,0,...,0,1,0,1,0,0,1,0,1,0
351,bhi95,95,Dan L,1,0,2,1,25,13823,0,...,0,1,0,1,0,0,1,0,1,0
352,bhi95,95,Dan L,1,0,3,1,14,7930,0,...,0,1,0,1,0,0,1,0,1,0
353,bhi95,95,Dan L,1,0,4,1,43,22885,0,...,0,1,0,1,0,0,1,0,1,0
